<a href="https://colab.research.google.com/github/bilmark0/Agile-Manufacturing-TDK-/blob/main/CV/CNN_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import random
import math
import datetime
import numpy as np
import PIL
import PIL.Image
import sys
from google.colab import drive, files
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt



In [2]:
# Upload the kaggle.json file
uploaded = files.upload()

# Move the uploaded file to the .kaggle directory
kaggle_api_path = os.path.expanduser("~/.kaggle")
if not os.path.exists(kaggle_api_path):
    os.makedirs(kaggle_api_path)

# Ensure the file is set with proper permissions
kaggle_json_path = next(iter(uploaded))  # Get the uploaded filename
os.rename(kaggle_json_path, f"{kaggle_api_path}/kaggle.json")
os.chmod(f"{kaggle_api_path}/kaggle.json", 0o600)

# Download the dataset from Kaggle
!kaggle datasets download -d markbilszky/agile-manufacturing-tdk --unzip

print("Dataset downloaded successfully.")

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/markbilszky/agile-manufacturing-tdk
License(s): unknown
100% 2.50G/2.50G [01:25<00:00, 30.9MB/s]
100% 2.50G/2.50G [01:25<00:00, 31.5MB/s]
Dataset downloaded successfully.


In [3]:
# Set your paths here
base_path = './'  # Replace with the directory containing Reference, error_2, and error_3
new_folder_path = os.path.join(base_path, 'training_data')

# Create the new folder
os.makedirs(new_folder_path, exist_ok=True)

# Move the folders
folders_to_move = ['Reference', 'error_2', 'error_3']
for folder_name in folders_to_move:
    shutil.move(os.path.join(base_path, folder_name), new_folder_path)

print("Folders moved successfully!")


Folders moved successfully!


In [4]:
# Path to the training data directory
data_dir = './training_data'

# List of main folders to check for nested duplicates
main_folders = os.listdir(data_dir)

for folder in main_folders:
    folder_path = os.path.join(data_dir, folder)

    # Check if this path is indeed a folder
    if os.path.isdir(folder_path):
        # Path to the nested duplicate folder (if exists)
        nested_folder_path = os.path.join(folder_path, folder)

        # Check if a nested folder with the same name exists
        if os.path.isdir(nested_folder_path):
            print(f"Found duplicate folder: {nested_folder_path}")

            # Move all files from nested folder to the main folder
            for filename in os.listdir(nested_folder_path):
                file_path = os.path.join(nested_folder_path, filename)
                target_path = os.path.join(folder_path, filename)

                # Move file to the main folder
                if os.path.isfile(file_path):
                    shutil.move(file_path, target_path)
                    #print(f"Moved {file_path} to {target_path}")

            # Remove the nested duplicate folder after moving its contents
            os.rmdir(nested_folder_path)
            print(f"Removed duplicate folder: {nested_folder_path}")

print("Duplicate folders cleaned up successfully.")

Found duplicate folder: ./training_data/Reference/Reference
Removed duplicate folder: ./training_data/Reference/Reference
Found duplicate folder: ./training_data/error_2/error_2
Removed duplicate folder: ./training_data/error_2/error_2
Found duplicate folder: ./training_data/error_3/error_3
Removed duplicate folder: ./training_data/error_3/error_3
Duplicate folders cleaned up successfully.


In [5]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [6]:
data_dir = './training_data'  # Ensure this path is correct for your dataset
all_classes = sorted(os.listdir(data_dir))  # List and sort class folders in the dataset directory
train_files = []


for cls in all_classes:
    cls_dir = os.path.join(data_dir, cls)
    if os.path.isdir(cls_dir):
        files = [os.path.join(cls_dir, file) for file in os.listdir(cls_dir) if file.endswith(('png', 'jpg', 'jpeg'))]
        print(f"Class: {cls}, Number of files: {len(files)}")
        train_files.extend(files)


Class: Reference, Number of files: 7000
Class: error_2, Number of files: 7903
Class: error_3, Number of files: 10000


In [ ]:
# Set a fixed random seed for reproducibility
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


# Sinusoidal learning rate callback
class SinusoidalLearningRate(Callback):
    def __init__(self, max_lr, min_lr, total_epochs):
        super(SinusoidalLearningRate, self).__init__()
        self.max_lr = max_lr
        self.min_lr = min_lr
        self.total_epochs = total_epochs

    def on_epoch_begin(self, epoch, logs=None):
        lr = self.min_lr + (self.max_lr - self.min_lr) * (0.5 * (1 + math.sin(2 * math.pi * epoch / self.total_epochs)))
        self.model.optimizer.learning_rate.assign(lr)
        print(f"Epoch {epoch + 1}: Learning rate is {lr:.6f}")

# Hyperparameters for the sinusoidal learning rate
max_lr = 0.01
min_lr = 0.001
total_epochs = 1000

# Use the sinusoidal learning rate in the callbacks
sinusoidal_lr_callback = SinusoidalLearningRate(max_lr=max_lr, min_lr=min_lr, total_epochs=50)

# Model setup
data_dir = './training_data'  # Ensure this path is correct for your dataset
img_height, img_width = 140, 190

'''
# Initialize the data generator
datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.4,
    horizontal_flip=True,
    validation_split=0.2  # Add validation split if needed
)
'''

# Split dataset into training, validation, and test sets
# This assumes that the data directory structure is correct
all_classes = sorted(os.listdir(data_dir))
train_files = []
val_files = []

# Collect all file paths and their corresponding classes
for cls in all_classes:
    cls_dir = os.path.join(data_dir, cls)
    if os.path.isdir(cls_dir):
        files = [os.path.join(cls_dir, file) for file in os.listdir(cls_dir) if file.endswith(('png', 'jpg', 'jpeg'))]
        train_files.extend(files)

# Create a split for training, validation, and test data
train_files, test_files = train_test_split(train_files, test_size=0.15, random_state=42)  # 20% for test
train_files, val_files = train_test_split(train_files, test_size=0.15, random_state=42)  # 20% of the remaining 80% for validation

# Function to create a generator from file paths
def create_generator(file_paths, batch_size):
    while True:
        np.random.shuffle(file_paths)
        for i in range(0, len(file_paths), batch_size):
            batch_paths = file_paths[i:i+batch_size]
            images = []
            labels = []
            for path in batch_paths:
                img = tf.keras.preprocessing.image.load_img(path, target_size=(img_height, img_width))
                img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Rescale
                images.append(img)
                label = path.split(os.path.sep)[-2]  # Get the label from the directory name
                labels.append(all_classes.index(label))
            yield np.array(images), tf.keras.utils.to_categorical(labels, num_classes=len(all_classes))

# Create the data generators
train_generator = create_generator(train_files, batch_size=32)
val_generator = create_generator(val_files, batch_size=32)
test_generator = create_generator(test_files, batch_size=32)

# Set up TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Define your model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    BatchNormalization(),
    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(64, activation='relu'),
    Dense(len(all_classes), activation='softmax')  # Update to number of classes
])

# Compile model with SGD optimizer and momentum
momentum = 0.9  # Set your momentum value
optimizer = SGD(learning_rate=max_lr, momentum=momentum)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Calculate the number of steps per epoch
steps_per_epoch = len(train_files) // 32
validation_steps = len(val_files) // 32
test_steps = len(test_files) // 32

# Train the model with the sinusoidal learning rate callback
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=total_epochs,
    callbacks=[early_stopping, model_checkpoint, sinusoidal_lr_callback, tensorboard_callback]
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator, steps=test_steps)
print(f'Test Loss: {loss}, Accuracy: {accuracy}')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1: Learning rate is 0.005500
Epoch 1/1000
562/562 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3998 - loss: 1.0873
Epoch 1: val_loss improved from inf to 1.08323, saving model to best_model.keras
562/562 ━━━━━━━━━━━━━━━━━━━━ 2013s 4s/step - accuracy: 0.3998 - loss: 1.0873 - val_accuracy: 0.4050 - val_loss: 1.0832
Epoch 2: Learning rate is 0.006064
Epoch 2/1000
562/562 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4182 - loss: 1.0744
Epoch 2: val_loss improved from 1.08323 to 1.02895, saving model to best_model.keras
562/562 ━━━━━━━━━━━━━━━━━━━━ 2013s 4s/step - accuracy: 0.4183 - loss: 1.0744 - val_accuracy: 0.4793 - val_loss: 1.0289
Epoch 3: Learning rate is 0.006619
Epoch 3/1000
562/562 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4415 - loss: 1.0483
Epoch 3: val_loss improved from 1.02895 to 0.97915, saving model to best_model.keras
562/562 ━━━━━━━━━━━━━━━━━━━━ 2025s 4s/step - accuracy: 0.4416 - loss: 1.0483 - val_accuracy: 0

In [ ]:
model.summary()

In [ ]:
print('Rétegek:\n')
for layer in model.layers:
    print("Réteg neve: ", layer.name, ', tanítható: ', layer.trainable)
    print(layer.get_config(),'\n')

In [ ]:
!pip3 install keras-visualizer
from keras_visualizer import visualizer
import matplotlib.image as mpimg
visualizer(model, file_format='png', view=True)
img = mpimg.imread('graph.png')
fig = plt.figure(figsize=[20,9])
plt.imshow(img)
plt.axis('off')

In [ ]:
logdir = "logs/fit/"
file_writer = tf.summary.create_file_writer(logdir)

# Use the test generator to get a batch of images
test_images, _ = next(test_generator)  # Get a batch of images and labels from the test generator

with tf.name_scope("Test") as scope:
    with file_writer.as_default():
        # Reshape the first three images from your test dataset
        img = np.reshape(test_images[0:3], (-1, img_height, img_width, 3))  # Adjust to (num_samples, height, width, channels)
        tf.summary.image("Test Images", img, step=0, max_outputs=3, description="Sample images from the test set")

# Remember to flush the writer to ensure the images are saved
file_writer.flush()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# Collect true labels and predictions from the test set
def collect_predictions_and_labels(generator_func, file_paths, steps):
    true_labels = []
    predictions = []

    # Create a new generator instance for predictions
    generator = generator_func(file_paths, batch_size=32)

    for _ in range(min(steps, len(file_paths) // 32)):  # Adjust to total batches
        x, y = next(generator)  # Get the next batch of images and labels
        preds = model.predict(x)  # Get predictions for the batch
        predictions.extend(np.argmax(preds, axis=1))  # Store predicted class indices
        true_labels.extend(np.argmax(y, axis=1))  # Store true class indices

    return true_labels, predictions

# Use the function to collect predictions
true_labels, predictions = collect_predictions_and_labels(create_generator, test_files, test_steps)

# Calculate the confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=all_classes)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Load TensorBoard
%load_ext tensorboard

# Start TensorBoard
logdir = "logs/fit/"  # Ensure this matches your logging path
%tensorboard --logdir {logdir}


In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot images with predicted and actual labels
def plot_images_with_labels(generator, model, num_images=9):
    # Get a batch of test images and labels
    test_images, test_labels = next(generator)

    # Predict the labels for the batch
    predictions = model.predict(test_images)
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(test_labels, axis=1)

    # Set up the plot grid
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(3, 3, i + 1)
        plt.imshow(test_images[i])
        actual_label = all_classes[actual_classes[i]]
        predicted_label = all_classes[predicted_classes[i]]
        plt.title(f"Actual: {actual_label}\nPredicted: {predicted_label}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Call the function to plot images and labels
plot_images_with_labels(test_generator, model, num_images=9)
